In [2]:
!pip install mysql-connector-python

  Using cached mysql_connector_python-9.4.0-cp313-cp313-win_amd64.whl.metadata (7.7 kB)
Using cached mysql_connector_python-9.4.0-cp313-cp313-win_amd64.whl (16.4 MB)


In [3]:
import sys
sys.executable

'C:\\pbl_dbms_project\\project_env\\Scripts\\python.exe'

In [4]:
pip show mysql-connector-python

Name: mysql-connector-python
Version: 9.4.0
Summary: A self-contained Python driver for communicating with MySQL servers, using an API that is compliant with the Python Database API Specification v2.0 (PEP 249).
Home-page: https://dev.mysql.com/doc/connector-python/en/
Author: Oracle and/or its affiliates
Author-email: 
License: GNU GPLv2 (with FOSS License Exception)
Location: C:\pbl_dbms_project\project_env\Lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector
import json
import subprocess
conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Shubham@01",
        database="expense_tracker "
    )
cursor = conn.cursor()
def get_table_schema(table_name):

    cursor.execute(f"DESCRIBE {table_name}")
    rows = cursor.fetchall()

    # Convert to structured dict
    schema = []
    for row in rows:
        schema.append({
            "field": row[0],
            "type": row[1],
            "null": row[2],
            "key": row[3],
            "default": row[4],
            "extra": row[5]
        })

    return json.dumps({table_name: schema}, indent=4)


def ask_agent(prompt):
    result = subprocess.run(
        ["ollama", "run", "llama3"],
         input=prompt.encode("utf-8"),
        capture_output=True
        
        )
    return result.stdout.decode("utf-8").strip()
schema=get_table_schema('transaction')
print(schema)
def chatbot():

    print("Bot: Hello! I will only decide if SQL can be generated or if I need to ask questions. (DB connected)")

    while True:
        user = input("You: ")
        if user.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            conn.close()
            break

        prompt = f"""
        You are a SQL assistant. You know this schema:
        {schema}

        Rules:
        1. If the user’s request is clear and enough info is available, output ONLY a valid SQL query.
        2. If something is missing (like table name, column names, or values), do not guess. 
           Instead, ask the user for clarification in plain English.
        3. Do not add explanations just SQL query or a clarification question.
        4. if someone ask for create table ask details and schema 

        User: {user}
        """
        reply = ask_agent(prompt)
        print("Bot:", reply)
        if reply.strip().lower().startswith(("select", "create", "insert", "update", "delete")):
            try:
                cursor.execute(reply)
                rows = cursor.fetchall()
                if rows:
                    print("Bot: Query Result →", rows)
                else:
                    print("Bot: Query executed successfully.")
                conn.commit()
            except Exception as e:
                print("Bot: SQL error →", e)

    conn.close()

if __name__ == "__main__":
    chatbot()

In [41]:
#ive refine it using ai and also add context memroy to it 

In [23]:
import subprocess
import mysql.connector
import json
import datetime

MAX_CONTEXT_CHARS = 3000  # limit to avoid exceeding LLM context window

# ------------------------
# LLM wrapper
# ------------------------
def ask_agent(prompt):
    result = subprocess.run(
        ["ollama", "run", "llama3"],
        input=prompt.encode("utf-8"),
        capture_output=True
    )
    return result.stdout.decode("utf-8").strip()

# ------------------------
# Safe serialization (fix for date/datetime)
# ------------------------
def serialize_value(val):
    if isinstance(val, (datetime.date, datetime.datetime)):
        return val.isoformat()
    return val

# ------------------------
# Fetch schema for table
# ------------------------
def get_table_schema(table_name, cursor):
    cursor.execute(f"DESCRIBE {table_name}")
    rows = cursor.fetchall()
    schema = []
    for row in rows:
        schema.append({
            "field": row[0],
            "type": row[1],
            "null": row[2],
            "key": row[3],
            "default": row[4],
            "extra": row[5]
        })
    return {table_name: schema}

# ------------------------
# Chatbot with memory
# ------------------------
def chatbot():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root1234",
        database="demo"
    )
    cursor = conn.cursor()

    schema = get_table_schema("transactions", cursor)
    history = []

    print("Bot: Hello! I remember context but will trim it if it gets too long. (DB connected)")

    while True:
        user = input("You: ")
        if user.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            break

        history.append(f"User: {user}")

        context = "\n".join(history)
        if len(context) > MAX_CONTEXT_CHARS:
            while len(context) > MAX_CONTEXT_CHARS and len(history) > 1:
                history.pop(0)
                context = "\n".join(history)

        # 🔹 Refined Prompt
        prompt = f"""
        You are a SQL assistant which is connected to database sever  and user you can only return 1 thing at time whether a message or a query dont mix both . You know this schema:
        {schema}

        Rules:
        1. If the user’s request is clear and enough info is available, output ONLY a valid SQL query no additional text.
        2. If something is missing (like table name, column names, or values), do not guess. 
           Instead, ask the user for clarification in plain English.
        3. Do not add explanations, just SQL query or a clarification question.
        4. If someone asks to create a table, request table name and schema details first.
        5. do not use (SQL + Text) then system will fail.
        6.whether you can return a text or a sql query dont mix.

        User: {user}
        """

        reply = ask_agent(prompt).strip()
        print("Bot:", reply)
        history.append(f"Bot: {reply}")

        if reply.lower().startswith(("select", "create", "insert", "update", "delete","where")):
            try:
                cursor.execute(reply)

                if reply.lower():
                    rows = cursor.fetchall()
                    col_names = [desc[0] for desc in cursor.description]
                    result_json = [dict(zip(col_names, [serialize_value(v) for v in row])) for row in rows]

                    analysis_prompt = f"""
                    You are an assistant analyzing SQL query results.

                    User asked: "{user}"
                    Query executed: {reply}
                    Query result (JSON): {json.dumps(result_json, indent=2)}

                    Based on this, answer the user clearly in natural language. Be concise.
                    """
                    analysis = ask_agent(analysis_prompt)
                    print("Bot:", analysis)
                    history.append(f"Bot: {analysis}")

                else:
                    conn.commit()
                    print("Bot: Query executed successfully.")

            except Exception as e:
                print("Bot: SQL error →", e)

                # 🔹 Retry: ask LLM to fix
                error_prompt = f"""
                The query failed with error: {str(e)}.
                User question: "{user}"
                SQL attempted: {reply}
                Suggest a corrected SQL query.
                """
                fixed_sql = ask_agent(error_prompt)
                print("Bot (retrying):", fixed_sql)

    conn.close()


In [ ]:
if __name__ == "__main__":
    chatbot()

Bot: Hello! I remember context but will trim it if it gets too long. (DB connected)


You:  hey


Bot: What would you like to do? Create a table, query existing data, or something else?


You:  on 2025-11-09  bike ride at maldevta  spend 100  category  6 ( add it to expense)


Bot: INSERT INTO transactions (date, amount, description, transaction_type, category_id) VALUES ('2025-11-09', '100.00', 'bike ride at Maldevta', 'EXPENSE', '6');
Bot: SQL error → 'NoneType' object is not iterable
Bot (retrying): The error message is indicating that one of the values being inserted into the database (in this case, the `date` value) is None, which cannot be iterated over. 

Based on your given question and SQL attempted, it seems like the issue might be with the date formatting. Python's datetime module doesn't accept dates in the 'yyyy-mm-dd' format directly.

Here's a corrected SQL query:

```
import datetime

date = datetime.datetime.strptime('2025-11-09', '%Y-%m-%d').date()
SQL attempted: INSERT INTO transactions (transaction_date, amount, description, transaction_type, category_id) VALUES (DATE(date), '100.00', 'bike ride at Maldevta', 'EXPENSE', '6');
```

This query will convert the given date string into a datetime object and then extract just the date part. The

You:  pls re add it


Bot: Please provide more information about what you would like to do with the "transactions" table. For example, would you like to insert, update, delete, or select data? If you have specific values in mind, please provide them as well.
